<a href="https://colab.research.google.com/github/GauravKanwat/CS6910_Assignment1/blob/main/CS6910_Assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.8/258.8 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.8 MB/s eta 0:00:00


In [1]:
import numpy as np
import pandas as pd
from tensorflow import keras
import matplotlib.pyplot as plt
from time import sleep
from tqdm import tqdm, trange
from sklearn.model_selection import train_test_split
import wandb
wandb.login()

ModuleNotFoundError: No module named 'wandb'

In [ ]:
#wandb.init(
    # set the wandb project where this run will be logged
#    project="DL-Assignment-1",

    # track hyperparameters and run metadata
#    config={
#    "eta": 0.02,
#    "architecture": "CNN",
#    "dataset": "CIFAR-100",
#    "epochs": 10,
#    }
#)

In [21]:
class NeuralNetwork:
    def __init__(self, x_input, y_input, num_of_pixels, hidden_neurons_list, num_hidden_layers, output_neurons):
      self.x_input = x_input
      self.y_input = y_input
      self.num_of_pixels = num_of_pixels
      self.hidden_neurons_list = hidden_neurons_list
      self.num_hidden_layers = num_hidden_layers
      self.output_neurons = output_neurons


    def initialize_parameters(self, num_of_pixels, hidden_neurons_list, num_hidden_layers, output_neurons):
        weights = {}
        biases = {}
        for l in range(num_hidden_layers):
          weights[l] = np.random.rand(hidden_neurons_list[l], num_of_pixels if l == 0 else hidden_neurons_list[l-1]) - 0.5
          biases[l] = np.random.rand(hidden_neurons_list[l], 1) - 0.5
        weights[num_hidden_layers] = np.random.rand(output_neurons, hidden_neurons_list[-1]) - 0.5
        biases[num_hidden_layers] = np.random.rand(output_neurons, 1) - 0.5
        return weights, biases

    def xavier_intialization(self, num_of_pixels, hidden_neurons_list, output_neurons):
        num_layers = len(hidden_neurons_list) + 1
        weights = {}
        biases = {}

        weights[0] = np.random.randn(hidden_neurons_list[0], num_of_pixels) * np.sqrt(1 / num_of_pixels)
        biases[0] = np.zeros((hidden_neurons_list[0], 1))

        # Initialize weights and biases for hidden layers
        for l in range(1, len(hidden_neurons_list)):
            weights[l] = np.random.randn(hidden_neurons_list[l], hidden_neurons_list[l-1]) * np.sqrt(1 / hidden_neurons_list[l-1])
            biases[l] = np.zeros((hidden_neurons_list[l], 1))

        # Initialize weights for last hidden layer to output layer
        weights[len(hidden_neurons_list)] = np.random.randn(output_neurons, hidden_neurons_list[-1]) * np.sqrt(1 / hidden_neurons_list[-1])
        biases[len(hidden_neurons_list)] = np.zeros((output_neurons, 1))
        return weights, biases

    def sigmoid(self, x):
        #return 1 / (1 + np.exp(-x))
        return 1 / (1 + np.exp(-np.clip(x, -500, 500)))

    def reLU(self, Z):
        return np.maximum(0, Z)

    def tanh(self, x):
      return np.tanh(x)

    def softmax(self, x):
        max_x = np.max(x, axis=0)
        exp_x = np.exp(x - max_x)  # avoiding overflow
        return exp_x / np.sum(exp_x, axis=0)

    def feedforward_propagation(self, X, weights, biases, num_hidden_layers, activation_function):
        a = {}
        h = {}

        for k in range(num_hidden_layers):
            if k == 0:
              a[k] = np.dot(weights[k], X) + biases[k]
              if(activation_function == "reLU"):
                h[k] = self.reLU(a[k])
              elif(activation_function == "sigmoid"):
                h[k] = self.sigmoid(a[k])
              elif(activation_function == "tanh"):
                h[k] = self.tanh(a[k])
            else:
              a[k] = np.dot(weights[k], h[k-1]) + biases[k]
              if(activation_function == "reLU"):
                h[k] = self.reLU(a[k])
              elif(activation_function == "sigmoid"):
                h[k] = self.sigmoid(a[k])
              elif(activation_function == "tanh"):
                h[k] = self.tanh(a[k])

        a[num_hidden_layers] = np.dot(weights[num_hidden_layers], h[num_hidden_layers - 1]) + biases[num_hidden_layers]
        y_hat = self.softmax(a[num_hidden_layers])
        return a, h, y_hat

    def one_hot(self, Y):
      if Y.max() != 9:
        one_hot_Y = np.zeros((Y.size, 10))
      else:
        one_hot_Y = np.zeros((Y.size, Y.max() + 1))
      one_hot_Y[np.arange(Y.size), Y] = 1
      one_hot_Y = one_hot_Y.T
      return one_hot_Y

    def deriv_sigmoid(self, Z):
      func = self.sigmoid(Z)
      return func * (1 - func)

    def deriv_reLU(self, Z):
      return Z > 0

    def deriv_tanh(self, x):
      sechX = 1 / np.cosh(x)
      return sechX ** 2

    def back_propagation(self, Y, fwd_A, fwd_H, weights, biases, pred_output, num_hidden_layers, activation_function):
      one_hot_Y = self.one_hot(Y)
      dA = {}
      dH = {}
      dW = {}
      dB = {}

      dA[num_hidden_layers] = pred_output - one_hot_Y

      for k in range(num_hidden_layers, 0, -1):
        dW[k] = np.dot(dA[k], fwd_H[k-1].T)
        dB[k] = np.mean(dA[k], axis=1, keepdims=True)

        dH[k-1] = np.dot(weights[k].T, dA[k])
        if(activation_function == "reLU"):
          dA[k-1] = np.multiply(dH[k-1], self.deriv_reLU(fwd_A[k-1]))
        elif(activation_function == "sigmoid"):
          dA[k-1] = np.multiply(dH[k-1], self.deriv_sigmoid(fwd_A[k-1]))
        elif(activation_function == "tanh"):
          dA[k-1] = np.multiply(dH[k-1], self.deriv_tanh(fwd_A[k-1]))
      return dW, dB

    def get_predictions(self, pred_output):
      return np.argmax(pred_output, axis = 0)

    def get_accuracy(self, y_pred, y_true):
      return np.sum(y_pred == y_true) / y_true.size

    def cross_entropy(self, y_pred, y_true):
      epsilon = 1e-15
      loss = -np.mean(np.sum(y_true * np.log(y_pred + epsilon), axis=0))
      return loss

    def gradient_descent(self, epochs, eta, activation_function, initialization):
      if(initialization == "normal"):
        weights, biases = self.initialize_parameters(self.num_of_pixels, self.hidden_neurons_list, self.num_hidden_layers, self.output_neurons)
      elif(initialization == "xavier"):
        weights, biases = self.xavier_intialization(self.num_of_pixels, self.hidden_neurons_list, self.output_neurons)

      for epoch in tqdm(range(epochs)):
        total_loss = 0
        fwd_a, fwd_h, pred_output = self.feedforward_propagation(self.x_input, weights, biases, self.num_hidden_layers, activation_function)
        del_w, del_b = self.back_propagation(self.y_input, fwd_a, fwd_h, weights, biases, pred_output, self.num_hidden_layers, activation_function)

        loss = self.cross_entropy(pred_output, self.y_input)

        # Update weights and biases
        for l in range(1, self.num_hidden_layers + 1):
          weights[l] -= eta * del_w[l]
          biases[l] -= eta * del_b[l]

        total_loss += loss

        if epoch % 10 == 0:
          accuracy = self.get_accuracy(self.get_predictions(pred_output), self.y_input)
          print(f"Iteration: {epoch}, Accuracy: {accuracy}, Loss: {total_loss}")
      return weights, biases

    def stochastic_gradient_descent(self, epochs, eta, activation_function, initialization, batch_size):
      if(initialization == "normal"):
        weights, biases = self.initialize_parameters(self.num_of_pixels, self.hidden_neurons_list, self.num_hidden_layers, self.output_neurons)
      elif(initialization == "xavier"):
        weights, biases = self.xavier_intialization(self.num_of_pixels, self.hidden_neurons_list, self.output_neurons)

      num_samples = self.x_input.shape[1]

      for epoch in tqdm(range(epochs)):
        total_loss = 0
        shuffled_indices = np.random.permutation(num_samples)
        for i in range(0, num_samples, batch_size):

          batch_indices = shuffled_indices[i:i+batch_size]
          x_batch = self.x_input[:, batch_indices]
          y_batch = self.y_input[batch_indices]

          fwd_a, fwd_h, pred_output = self.feedforward_propagation(x_batch, weights, biases, self.num_hidden_layers, activation_function)
          del_w, del_b = self.back_propagation(y_batch, fwd_a, fwd_h, weights, biases, pred_output, self.num_hidden_layers, activation_function)

          loss = self.cross_entropy(pred_output, self.y_input)

          # Update weights and biases
          for l in range(1, self.num_hidden_layers + 1):
            weights[l] -= eta * del_w[l]
            biases[l] -= eta * del_b[l]

        total_loss += loss

        if epoch % 1 == 0:
          accuracy = self.get_accuracy(self.get_predictions(pred_output), y_batch)
          print(f"Iteration: {epoch}, Accuracy: {accuracy}, Loss: {total_loss}")

      return weights, biases


    def momentum_based_gradient_descent(self, epochs, eta, beta, activation_function, initialization):
      if(initialization == "normal"):
        weights, biases = self.initialize_parameters(self.num_of_pixels, self.hidden_neurons_list, self.num_hidden_layers, self.output_neurons)
      elif(initialization == "xavier"):
        weights, biases = self.xavier_intialization(self.num_of_pixels, self.hidden_neurons_list, self.output_neurons)

      prev_uw = {}
      prev_ub = {}

      for l in range(1, self.num_hidden_layers + 1):
        prev_uw[l] = 0
        prev_ub[l] = 0

      for epoch in tqdm(range(epochs)):
        total_loss = 0
        fwd_a, fwd_h, pred_output = self.feedforward_propagation(self.x_input, weights, biases, self.num_hidden_layers, activation_function)
        del_w, del_b = self.back_propagation(self.y_input, fwd_a, fwd_h, weights, biases, pred_output, self.num_hidden_layers, activation_function)

        loss = self.cross_entropy(pred_output, self.y_input)

        # Update weights and biases
        for l in range(1, self.num_hidden_layers + 1):
          uw = beta * prev_uw[l] + eta * del_w[l]
          ub = beta * prev_ub[l] + eta * del_b[l]
          weights[l] -= uw
          biases[l] -= ub
          prev_uw[l] = uw
          prev_ub[l] = ub

          total_loss = loss

        if epoch % 10 == 0:
          accuracy = self.get_accuracy(self.get_predictions(pred_output), self.y_input)
          print(f"Iteration: {epoch}, Accuracy: {accuracy}, Loss: {total_loss}")
      return weights, biases

    def nesterov_accelerated_gradient_descent(self, epochs, eta, beta, activation_function, initialization):
      if(initialization == "normal"):
        weights, biases = self.initialize_parameters(self.num_of_pixels, self.hidden_neurons_list, self.num_hidden_layers, self.output_neurons)
      elif(initialization == "xavier"):
        weights, biases = self.xavier_intialization(self.num_of_pixels, self.hidden_neurons_list, self.output_neurons)

      prev_vw = 0
      prev_vb = 0

      for epoch in tqdm(range(epochs)):

        fwd_a, fwd_h, pred_output = self.feedforward_propagation(self.x_input, weights, biases, self.num_hidden_layers, activation_function)
        del_w, del_b = self.back_propagation(self.y_input, fwd_a, fwd_h, weights, biases, pred_output, self.num_hidden_layers, activation_function)

        v_w = beta*prev_vw
        v_b = beta*prev_vb

        # Update weights and biases
        for l in range(1, self.num_hidden_layers + 1):
          vw = beta * prev_vw + eta * del_w[l]
          vb = beta * prev_vb + eta * del_b[l]
          weights[l] -= vw
          biases[l] -= vb
          prev_uw = vw
          prev_ub = vb

        if epoch % 10 == 0:
          accuracy = self.get_accuracy(self.get_predictions(pred_output), self.y_input)
          print(f"Iteration: {epoch}, Accuracy: {accuracy}")
      return weights, biases

    def adagrad_gradient_descent(self, epochs, eta, eps, activation_function, initialization):
      if(initialization == "normal"):
        weights, biases = self.initialize_parameters(self.num_of_pixels, self.hidden_neurons_list, self.num_hidden_layers, self.output_neurons)
      elif(initialization == "xavier"):
        weights, biases = self.xavier_intialization(self.num_of_pixels, self.hidden_neurons_list, self.output_neurons)

      v_w = {}
      v_b = {}

      for l in range(1, self.num_hidden_layers + 1):
        v_w[l] = 0
        v_b[l] = 0

      for epoch in tqdm(range(epochs)):
        fwd_a, fwd_h, pred_output = self.feedforward_propagation(self.x_input, weights, biases, self.num_hidden_layers, activation_function)
        del_w, del_b = self.back_propagation(self.y_input, fwd_a, fwd_h, weights, biases, pred_output, self.num_hidden_layers, activation_function)

        # Update weights and biases
        for l in range(1, self.num_hidden_layers + 1):
          v_w[l] = v_w[l] + del_w[l]**2
          v_b[l] = v_b[l] + del_b[l]**2

          weights[l] -= eta * del_w[l] / (np.sqrt(v_w[l]) + eps)
          biases[l] -= eta * del_b[l] / (np.sqrt(v_b[l]) + eps)

        if epoch % 10 == 0:
          accuracy = self.get_accuracy(self.get_predictions(pred_output), self.y_input)
          print(f"Iteration: {epoch}, Accuracy: {accuracy}")
      return weights, biases

    def rmsProp_gradient_descent(self, epochs, eta, eps, beta, activation_function, initialization):
      if(initialization == "normal"):
        weights, biases = self.initialize_parameters(self.num_of_pixels, self.hidden_neurons_list, self.num_hidden_layers, self.output_neurons)
      elif(initialization == "xavier"):
        weights, biases = self.xavier_intialization(self.num_of_pixels, self.hidden_neurons_list, self.output_neurons)

      v_w = {}
      v_b = {}

      for l in range(1, self.num_hidden_layers + 1):
        v_w[l] = 0
        v_b[l] = 0

      for epoch in tqdm(range(epochs)):
        fwd_a, fwd_h, pred_output = self.feedforward_propagation(self.x_input, weights, biases, self.num_hidden_layers, activation_function)
        del_w, del_b = self.back_propagation(self.y_input, fwd_a, fwd_h, weights, biases, pred_output, self.num_hidden_layers, activation_function)

        # Update weights and biases
        for l in range(1, self.num_hidden_layers + 1):
          v_w[l] = (beta * v_w[l]) + ((1-beta) * del_w[l] ** 2)
          v_b[l] = (beta * v_b[l]) + ((1-beta) * del_b[l] ** 2)

          weights[l] -= eta * del_w[l] / (np.sqrt(v_w[l]) + eps)
          biases[l] -= eta * del_b[l] / (np.sqrt(v_b[l]) + eps)

        if epoch % 10 == 0:
          accuracy = self.get_accuracy(self.get_predictions(pred_output), self.y_input)
          print(f"Iteration: {epoch}, Accuracy: {accuracy}")
      return weights, biases

    def adam_gradient_descent(self, epochs, eta, eps, beta1, beta2, activation_function, initialization):
      if(initialization == "normal"):
        weights, biases = self.initialize_parameters(self.num_of_pixels, self.hidden_neurons_list, self.num_hidden_layers, self.output_neurons)
      elif(initialization == "xavier"):
        weights, biases = self.xavier_intialization(self.num_of_pixels, self.hidden_neurons_list, self.output_neurons)

      print(self.x_input.shape)

      m_w = {}
      m_b = {}
      v_w = {}
      v_b = {}
      m_w_hat = {}
      m_b_hat = {}
      v_w_hat = {}
      v_b_hat = {}

      for l in range(1, self.num_hidden_layers + 1):
        m_w[l] = 0
        m_b[l] = 0
        v_w[l] = 0
        v_b[l] = 0
        m_w_hat[l] = 0
        m_b_hat[l] = 0
        v_w_hat[l] = 0
        v_b_hat[l] = 0

      for epoch in tqdm(range(epochs)):
        fwd_a, fwd_h, pred_output = self.feedforward_propagation(self.x_input, weights, biases, self.num_hidden_layers, activation_function)
        del_w, del_b = self.back_propagation(self.y_input, fwd_a, fwd_h, weights, biases, pred_output, self.num_hidden_layers, activation_function)

        # Update weights and biases
        for l in range(1, self.num_hidden_layers + 1):
          m_w[l] = (beta1 * m_w[l]) + (1-beta1) * del_w[l]
          m_b[l] = (beta1 * m_b[l]) + (1-beta1) * del_b[l]

          v_w[l] = beta2 * v_w[l] + (1 - beta2) * (del_w[l] ** 2)
          v_b[l] = beta2 * v_b[l] + (1 - beta2) * (del_b[l] ** 2)

          m_w_hat[l] = m_w[l]/(1-np.power(beta1, l+1))
          m_b_hat[l] = m_b[l]/(1-np.power(beta1, l+1))
          v_w_hat[l] = v_w[l]/(1-np.power(beta2, l+1))
          v_b_hat[l] = v_b[l]/(1-np.power(beta2, l+1))

          #update parameters
          weights[l] -= eta*m_w_hat[l]/(np.sqrt(v_w_hat[l])+eps)
          biases[l] -= eta*m_b_hat[l]/(np.sqrt(v_b_hat[l])+eps)

        if epoch % 10 == 0:
          accuracy = self.get_accuracy(self.get_predictions(pred_output), self.y_input)
          print(f"Iteration: {epoch}, Accuracy: {accuracy}")
      return weights, biases


def main():

  fashion_mnist = keras.datasets.fashion_mnist
  (x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
  x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=10000, random_state=42)
  classes = {0:"T-shirt/top", 1:"Trouser", 2:"Pullover", 3:"Dress", 4:"Coat", 5:"Sandal", 6:"Shirt", 7:"Sneaker", 8:"Bag", 9:"Ankle Boot"}
  x_train_norm = x_train / 255
  x_test_norm = x_test / 255

  # Define hyperparameters
  num_of_pixels = 28 * 28                                                         #28 * 28 = 784 pixels
  hidden_neurons_list = [128]
  num_hidden_layers = len(hidden_neurons_list)
  output_neurons = 10
  eta = 1e-5
  epochs = 51
  activation_function = "sigmoid"
  initialization = "normal"
  batch_size = 1
  beta = 0.5
  beta1 = 0.9
  beta2 = 0.999
  eps = 1e-10

  #Taking pixels as inputs
  x_train_input = x_train_norm.reshape(len(x_train_norm), num_of_pixels)                      #flattening the image into 1d array
  x_test_input = x_test_norm.reshape(len(x_test_norm), num_of_pixels)                         #same thing
  x_train_input = x_train_input.T
  x_test_input = x_test_input.T

  # Neural network class -> nn object
  nn = NeuralNetwork(x_train_input, y_train, num_of_pixels, hidden_neurons_list, num_hidden_layers, output_neurons)

  # Call the gradient_descent method
  #weights, biases = nn.gradient_descent(epochs, eta, activation_function, initialization)
  #weights, biases = nn.stochastic_gradient_descent(epochs, eta, activation_function, initialization, batch_size)
  weights, biases = nn.momentum_based_gradient_descent(epochs, eta, beta, activation_function, initialization)
  #weights, biases = nn.nesterov_accelerated_gradient_descent(epochs, eta, beta, activation_function, initialization)
  #weights, biases = nn.adagrad_gradient_descent(epochs, eta, eps, activation_function, initialization)
  #weights, biases = nn.rmsProp_gradient_descent(epochs, eta, eps, beta, activation_function, initialization)
  #weights, biases = nn.adam_gradient_descent(epochs, eta, eps, beta1, beta2, activation_function, initialization)

if __name__ == "__main__":
    main()

  2%|▏         | 1/51 [00:01<00:53,  1.07s/it]

Iteration: 0, Accuracy: 0.1019, Loss: 176.96867275650573


 22%|██▏       | 11/51 [00:08<00:31,  1.28it/s]

Iteration: 10, Accuracy: 0.58074, Loss: 144.4914034596916


 41%|████      | 21/51 [00:17<00:24,  1.23it/s]

...

Iteration: 30, Accuracy: 0.72114, Loss: 179.8014131210799


 80%|████████  | 41/51 [00:34<00:07,  1.29it/s]

Iteration: 40, Accuracy: 0.73464, Loss: 190.7055024683113


100%|██████████| 51/51 [00:42<00:00,  1.19it/s]

Iteration: 50, Accuracy: 0.74292, Loss: 199.20920827882202
